In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import datetime
from geographiclib.geodesic import Geodesic
plt.rcParams['figure.max_open_warning'] = 0

In [2]:
#加载训练数据
train_dataset = pd.read_csv("train.csv",parse_dates = [["date","time"]])

In [3]:
#1.靠近岸边
del1 = [305808, 717484, 843052, 803514, 996179, 477271, 370548, 460277,768635]
#2.长度不足
del2 = [451328,742548,640706,65874,996179,831336,316790]
#3.断点
del3 = [607240]
del_all = list(set(del1).union(set(del2)).union(set(del3)))

In [4]:
#筛选数据集
train_dataset = train_dataset[~train_dataset.id.isin(del_all)]

In [5]:
def GPS_filter(data,lon,lat,num):
    dis_step = data["lat"].diff().abs().mean().mean()
    distance = 0.5*dis_step*(num**0.5)
    GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
    while GPSfilter.sum().sum()!=num:
        if GPSfilter.sum().sum()>num:
            max_dis = [data["lat"][GPSfilter].max().max()-lat,
                        data["lon"][GPSfilter].max().max()-lon,
                        lat-data["lat"][GPSfilter].min().min(),
                        lon-data["lon"][GPSfilter].min().min()]
            distance = max(max_dis)
            GPSfilter_range = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
            GPSfilter_range = GPSfilter ^ GPSfilter_range
            GPSfilter_range_num = len(np.where(GPSfilter_range)[0])          
            if GPSfilter_range_num>1 and (GPSfilter.sum().sum()-num)<GPSfilter_range_num:
                range_num = num+GPSfilter_range_num-GPSfilter.sum().sum()
                GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
                coord = np.where(GPSfilter_range)
                #print(range_num)  
                for i in range(range_num):
                    GPSfilter.iloc[coord[0][i], coord[1][i]] = True
            else:
                    GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
        elif GPSfilter.sum().sum()<num:
            distance = distance +dis_step*0.0999
            GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
        #print(GPSfilter.sum().sum())
    return GPSfilter

In [6]:
#风场
wd_u_avg = []
wd_v_avg = []
from tqdm import tqdm
for index, row in tqdm(train_dataset.iterrows()):
    date_temp = row.date_time + datetime.timedelta(hours=-20) #风场
    hour_temp = date_temp.hour
    if date_temp.strftime("%Y%m%d") not in ["20191221","20200319"]:
        wd=Dataset('wind/wind_hour_%s12.nc' % date_temp.strftime("%Y%m%d"))
    else:
        date_temp = date_temp + datetime.timedelta(hours=-24)
        hour_temp = date_temp.hour + 24
        wd=Dataset('wind/wind_hour_%s12.nc' % date_temp.strftime("%Y%m%d"))
    wd_lon = wd.variables['LONGITUDE1_151'][:].data
    wd_lat = wd.variables['LATITUDE1_151'][:].data
    wd_u = wd.variables['U10'][:].data[hour_temp]
    wd_v = wd.variables['V10'][:].data[hour_temp]
    wd_db = pd.DataFrame(np.zeros(((wd_u.shape[0], wd_u.shape[1]*4))),index=list(range(wd_u.shape[0])),columns=[["lat"]*wd_u.shape[1]+["lon"]*wd_u.shape[1]+["u"]*wd_u.shape[1]+["v"]*wd_u.shape[1],list(range(wd_u.shape[1]))*4])
    wd_db["lat"] = wd_lat
    wd_db["lon"] = wd_lon
    wd_db["lon"] = wd_db["lon"].T
    wd_db["u"] = wd_u
    wd_db["v"] = wd_v
    GPS_sub4 = GPS_filter(wd_db,row.lng,row.lat,4)
    dis_weights = []
    for i in zip(wd_db["lon"][GPS_sub4].unstack().dropna(how='all'),wd_db["lat"][GPS_sub4].unstack().dropna(how='all')):
        dis_weights.append(1/Geodesic.WGS84.Inverse(row.lat,row.lng,i[1], i[0])["s12"])
    wd_u_avg.append(np.average(wd_db["u"][GPS_sub4].unstack().dropna(how='all'),weights = dis_weights))
    wd_v_avg.append(np.average(wd_db["v"][GPS_sub4].unstack().dropna(how='all'),weights = dis_weights))

54720it [1:35:43,  9.53it/s]


In [7]:
train_dataset["wd_u_avg"] = wd_u_avg
train_dataset["wd_v_avg"] = wd_v_avg

In [8]:
train_dataset.to_csv("temp1.csv")

In [9]:
#洋流
Oflow_u_avg = []
Oflow_v_avg = []
from tqdm import tqdm
for index, row in tqdm(train_dataset.iterrows()):
    date_temp = row.date_time + datetime.timedelta(hours=-8) #洋流
    hour_temp = date_temp.hour
    Oflow=Dataset('current/current_hour_%s.nc' % date_temp.strftime("%Y%m%d"))
    Oflow_lon = Oflow.variables['lon'][:].data.T  # 经度间隔0.005度
    Oflow_lat = Oflow.variables['lat'][:].data.T   # 纬度间隔0.01度
    Oflow_u = Oflow.variables['u'][:].data[hour_temp][0].T
    Oflow_v = Oflow.variables['v'][:].data[hour_temp][0].T
    Oflow_db = pd.DataFrame(np.zeros(((Oflow_u.shape[0], Oflow_u.shape[1]*4))),index=list(range(Oflow_u.shape[0])),columns=[["lat"]*Oflow_u.shape[1]+["lon"]*Oflow_u.shape[1]+["u"]*Oflow_u.shape[1]+["v"]*Oflow_u.shape[1],list(range(Oflow_u.shape[1]))*4])
    Oflow_db["lat"] = Oflow_lat
    Oflow_db["lon"] = Oflow_lon
    Oflow_db["u"] = Oflow_u
    Oflow_db["v"] = Oflow_v
    
    GPS_sub9 = GPS_filter(Oflow_db,row.lng,row.lat,4)
    dis_weights = []
    for i in zip(Oflow_db["lon"][GPS_sub9].unstack().dropna(how='all'),Oflow_db["lat"][GPS_sub9].unstack().dropna(how='all')):
        dis_weights.append(1/Geodesic.WGS84.Inverse(row.lat,row.lng,i[1], i[0])["s12"])
    Oflow_u_avg.append(np.average(Oflow_db["u"][GPS_sub9].unstack().dropna(how='all'),weights = dis_weights))
    Oflow_v_avg.append(np.average(Oflow_db["v"][GPS_sub9].unstack().dropna(how='all'),weights = dis_weights))

54720it [4:28:25,  3.40it/s]


In [10]:
train_dataset["Oflow_u_avg"] = Oflow_u_avg
train_dataset["Oflow_v_avg"] = Oflow_v_avg

In [11]:
train_dataset.to_csv("temp2.csv")

In [12]:
#位移
train_dataset.reset_index(inplace = True)
move_u = []
move_v = []
for i in range(train_dataset.shape[0]-1):
    if train_dataset.id[i] == train_dataset.id[i+1]:
        distance = Geodesic.WGS84.Inverse(train_dataset.lat[i],train_dataset.lng[i],train_dataset.lat[i+1],train_dataset.lng[i+1])
        move_u.append(distance["s12"] * math.sin(math.radians(distance["azi1"]))) #move_u: 位移东向向量
        move_v.append(distance["s12"] * math.cos(math.radians(distance["azi1"]))) #move_v: 位移北向向量
    else:
        move_u.append(np.nan)
        move_v.append(np.nan)
move_u.append(np.nan)
move_v.append(np.nan)    

In [13]:
train_dataset["move_u"] = move_u
train_dataset["move_v"] = move_v

In [14]:
train_dataset.set_index("index",inplace = True)

In [15]:
train_dataset.to_csv("temp3.csv")